In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from  collections import Counter
from datetime import datetime

In [ ]:
netflix_df = pd.read_csv('netflix_titles.csv', encoding='latin-1')

In [ ]:
netflix_df.head()

In [ ]:
summary = netflix_df.describe()
print(summary)

In [ ]:
netflix_df.info()

In [ ]:
netflix_df.isnull().sum()

In [ ]:
empty_director = netflix_df[netflix_df['director'].isnull()]
print(empty_director)

In [ ]:
cols = ['director', 'cast', 'country']

netflix_df[cols] = netflix_df[cols].fillna('Unknown')
print(netflix_df[cols])
# fill in  mode
netflix_df['date_added'].fillna(netflix_df['date_added'].mode()[0],inplace=True)
netflix_df['rating'].fillna(netflix_df['rating'].mode()[0],inplace=True)
for i in netflix_df.index:
    if netflix_df.loc[i, 'rating']== 'UR':
        netflix_df.loc[i,'rating']= 'NR'
#drop row with missing values
netflix_df=netflix_df.dropna(subset=['duration'])



In [ ]:
netflix_df.head()

In [ ]:
# Check for duplicate rows
duplicate_rows = netflix_df[netflix_df.duplicated()]

print(duplicate_rows)

In [ ]:
netflix_df = netflix_df.iloc[0:8808:]

In [ ]:
netflix_df.drop_duplicates()

In [ ]:
# Create separate DataFrames for TV Shows and Movies
tv_shows_df = netflix_df[netflix_df['type'] == 'TV Show']
print(tv_shows_df)

In [ ]:
movies_df = netflix_df[netflix_df['type'] == 'Movie']
print(movies_df)

In [ ]:
# transfer to datetime type
netflix_df['duration_mins']=netflix_df[netflix_df['type']=='Movie']['duration'].str.extract('(\d+)').astype(float)
netflix_df['duration_seasons']=netflix_df[netflix_df['type']=='TV Show']['duration'].str.extract('(\d+)').astype(float)
#NaN values replace with 0
netflix_df[['duration_mins', 'duration_seasons']]= netflix_df[['duration_mins', 'duration_seasons']].fillna(0)
#drop the  duration column
netflix_df=netflix_df.drop('duration', axis=1)


In [ ]:
netflix_df.head()